<a href="https://colab.research.google.com/github/dsfdev2023/studentEvaluatiuon/blob/main/HR_KPIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1 & 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 3: Read the Excel file
import pandas as pd

file_path = '/content/drive/MyDrive/1KPIS HR /updated_file.xlsx' # Change to your file path
df = pd.read_excel(file_path)


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [ ]:
# Step 4: Preprocess the data
def convert_to_minutes(time_val):
    if pd.isnull(time_val):
        return 0

    # Check if the value is an integer
    if isinstance(time_val, int):
        return time_val

    # Check if the value is a string
    elif isinstance(time_val, str):
        hours, minutes = map(int, time_val.split(':'))
        return hours * 60 + minutes

    # For datetime.time objects
    elif hasattr(time_val, 'hour'):
        return time_val.hour * 60 + time_val.minute

    # Fallback for any other type
    else:
        return 0

df['real Presence'] = df['real Presence'].apply(convert_to_minutes)
df['Pause'] = df['Pause'].apply(convert_to_minutes)

# Step 5: Perform aggregations
grouped = df.groupby('Ligne').agg({
    'real Presence': 'sum',
    'Breaks': 'sum',
    'Pause': 'sum'
})

# Import Plotly
import plotly.express as px

# Ensure that your data frame 'grouped' is reset to include 'Ligne' as a column
grouped = grouped.reset_index()

# Real Presence by Ligne - Interactive Graph with Different Colors
fig_real_presence = px.bar(grouped, x='Ligne', y='real Presence', color='Ligne',
                           labels={'real Presence': 'Real Presence (Minutes)'},
                           title='Sum of Real Presence by Ligne')
fig_real_presence.show()

# Breaks by Ligne - Interactive Graph with Different Colors
fig_breaks = px.bar(grouped, x='Ligne', y='Breaks', color='Ligne',
                    labels={'Breaks': 'Breaks (Count)'},
                    title='Sum of Breaks by Ligne')
fig_breaks.show()

# Pause by Ligne - Interactive Graph with Different Colors
fig_pause = px.bar(grouped, x='Ligne', y='Pause', color='Ligne',
                   labels={'Pause': 'Pause (Minutes)'},
                   title='Sum of Pause by Ligne')
fig_pause.show()


# **GRAPHS FOR 'BAKER' LIGNE**

In [ ]:
# Step 1: Filter the DataFrame for 'Ligne' equal to 'BAKER'
df_baker = df[df['Ligne'] == 'BAKER']

# Step 2: Create Histograms

# Pause Histogram
fig_pause_baker = px.histogram(df_baker, x='Matricule', y='Pause',
                               title='Pause Time for BAKER Matricule',
                               labels={'Pause': 'Pause (Minutes)'},
                               category_orders={"Matricule": df_baker.sort_values(by='Pause', ascending=False)['Matricule'].unique()})
fig_pause_baker.update_layout(bargap=0.2)
fig_pause_baker.show()

# Breaks Histogram
fig_breaks_baker = px.histogram(df_baker, x='Matricule', y='Breaks',
                                title='Breaks Count for BAKER Matricule',
                                labels={'Breaks': 'Breaks (Count)'},
                                category_orders={"Matricule": df_baker.sort_values(by='Breaks', ascending=False)['Matricule'].unique()})
fig_breaks_baker.update_layout(bargap=0.2)
fig_breaks_baker.show()

# Real Presence Histogram
fig_real_presence_baker = px.histogram(df_baker, x='Matricule', y='real Presence',
                                       title='Real Presence Time for BAKER Matricule',
                                       labels={'real Presence': 'Real Presence (Minutes)'},
                                       category_orders={"Matricule": df_baker.sort_values(by='real Presence', ascending=False)['Matricule'].unique()})
fig_real_presence_baker.update_layout(bargap=0.2)
fig_real_presence_baker.show()



# **FILTER BY SHIFT**

In [ ]:
# Filter the DataFrame for 'Ligne' equal to 'BAKER'
df_baker = df[df['Ligne'] == 'BAKER']

# Function to create histogram for each metric
def create_histogram(df, metric, title, label):
    # Create a histogram for each 'Shift'
    fig = px.histogram(df, x='Matricule', y=metric, facet_col='Shift', color='Shift',
                       title=title, labels={metric: label},
                       category_orders={"Shift": df['Shift'].unique()})
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.update_layout(bargap=0.2)
    return fig

# Pause Histogram
fig_pause_baker = create_histogram(df_baker, 'Pause',
                                   f'Pause Time for BAKER Matricule (Total Matricules: {df_baker["Matricule"].nunique()})',
                                   'Pause (Minutes)')
fig_pause_baker.show()

# Real Presence Histogram
fig_real_presence_baker = create_histogram(df_baker, 'real Presence',
                                           f'Real Presence Time for BAKER Matricule (Total Matricules: {df_baker["Matricule"].nunique()})',
                                           'Real Presence (Minutes)')
fig_real_presence_baker.show()

# Breaks Histogram
fig_breaks_baker = create_histogram(df_baker, 'Breaks',
                                    f'Breaks Count for BAKER Matricule (Total Matricules: {df_baker["Matricule"].nunique()})',
                                    'Breaks (Count)')
fig_breaks_baker.show()


# **Table Summary**

We will include "Matricules" of employees who have:
* Over 120 minutes of "Pause", or
* Over 2 "Breaks", or
* Less than 600 minutes of "real Presence",

But we will exclude those who have:
* More than 600 minutes of "real Presence" and fewer than 4 "Breaks".

In [ ]:
# Filter the DataFrame for 'Ligne' equal to 'BAKER'
df_baker = df[df['Ligne'] == 'BAKER']

# Calculate the sums of 'Pause', 'Breaks', and 'real Presence' for each 'Matricule'
aggregated_data = df_baker.groupby('Matricule').agg({
    'Pause': 'sum',
    'Breaks': 'sum',
    'real Presence': 'sum'
}).reset_index()

# Convert 'Pause' and 'real Presence' from the hour:minute format to total minutes
aggregated_data['Pause'] = aggregated_data['Pause'].apply(convert_to_minutes)
aggregated_data['real Presence'] = aggregated_data['real Presence'].apply(convert_to_minutes)

# Conditions for filtering
condition = (
    (aggregated_data['Pause'] > 120) |
    (aggregated_data['Breaks'] > 2) |
    (aggregated_data['real Presence'] < 600)
) & ~(
    (aggregated_data['real Presence'] > 600) &
    (aggregated_data['Breaks'] < 4)
)

# Display the table
filtered_matricules = aggregated_data[condition]

sorted_filtered_matricules = filtered_matricules.sort_values(by='real Presence', ascending=True)


# Count the number of matricules in the sorted filtered table
num_matricules = sorted_filtered_matricules.shape[0]

# Print the number of matricules
print(f"Number of Employees in the table: {num_matricules}")

# Display the sorted table
sorted_filtered_matricules

Number of Employees in the table: 20


,Matricule,Pause,Breaks,real Presence
12,0599,465,6,250
18,2212,17,2,295
27,6861,115,4,344
40,7069,322,5,397
7,0432,326,8,405
28,6913,202,4,507
38,7032,183,3,529
26,6807,179,6,536
15,2200,158,3,553
17,2211,140,3,554


## 10 "Matricules" with the lowest "real Presence" and the other with the 10 "Matricules" with the highest "real Presence":

In [ ]:
from IPython.display import display

# Define a function to convert minutes to "HH:MM" format
def minutes_to_hours(minutes):
    hours = minutes // 60
    minutes = minutes % 60
    return f"{hours:02d}:{minutes:02d}"

# Group by 'Matricule' and sum 'real Presence' for each 'Matricule'
aggregated_baker = df_baker.groupby('Matricule').agg({
    'real Presence': 'sum'
}).reset_index()

# Assuming 'real Presence' is already in minutes as per previous context
# If 'real Presence' is in hour:minute format, uncomment the following line
# aggregated_baker['real Presence'] = aggregated_baker['real Presence'].apply(convert_to_minutes)

# Convert 'real Presence' to "HH:MM" format and create a new column
aggregated_baker['real Presence (HH:MM)'] = aggregated_baker['real Presence'].apply(minutes_to_hours)

# Get the 10 matricules with the lowest 'real Presence'
lowest_real_presence_baker = aggregated_baker.nsmallest(10, 'real Presence')

# Get the 10 matricules with the highest 'real Presence'
highest_real_presence_baker = aggregated_baker.nlargest(10, 'real Presence')

# Display the tables with nicer formatting
print("10 Matricules with the Lowest Real Presence:")
display(lowest_real_presence_baker[['Matricule', 'real Presence', 'real Presence (HH:MM)']])

print("\n10 Matricules with the Highest Real Presence:")
display(highest_real_presence_baker[['Matricule', 'real Presence', 'real Presence (HH:MM)']])


10 Matricules with the Lowest Real Presence:


,Matricule,real Presence,real Presence (HH:MM)
12,0599,250,04:10
18,2212,295,04:55
27,6861,344,05:44
40,7069,397,06:37
7,0432,405,06:45
28,6913,507,08:27
38,7032,529,08:49
26,6807,536,08:56
15,2200,553,09:13
17,2211,554,09:14



10 Matricules with the Highest Real Presence:


,Matricule,real Presence,real Presence (HH:MM)
10,0508,721,12:01
21,2304,719,11:59
19,2219,712,11:52
33,6950,700,11:40
20,2223,698,11:38
14,2189,693,11:33
24,6627,690,11:30
47,7238,684,11:24
2,0056,679,11:19
8,0451STG,677,11:17


## Difference between "real Presence" and 600 minutes for all "Matricules" under the "BAKER" line

In [ ]:
# Assuming df is your original DataFrame
# Filter the DataFrame for 'Ligne' equal to 'BAKER'
df_baker = df[df['Ligne'] == 'BAKER'].copy()

# Define a function to convert "HH:MM" format to minutes
def convert_to_minutes(time_str):
    if isinstance(time_str, str):  # Check if the input is a string
        hours, minutes = map(int, time_str.split(':'))
        return hours * 60 + minutes
    else:
        return time_str  # If it's not a string, return as is

# Convert 'real Presence' from "HH:MM" format to minutes before the aggregation
df_baker['real Presence Minutes'] = df_baker['real Presence'].apply(convert_to_minutes)

# Now, group by 'Matricule' and sum 'real Presence Minutes' for each 'Matricule'
aggregated_baker = df_baker.groupby('Matricule').agg({
    'real Presence Minutes': 'sum'
}).reset_index()

# Define a function to calculate the positive difference only if 'real Presence' is less than 600 minutes
def calculate_positive_difference(real_presence_minutes):
    return max(600 - real_presence_minutes, 0)

# Define a function to convert minutes to hours and minutes format
def minutes_to_hours(minutes):
    hours = minutes // 60
    minutes = minutes % 60
    return f"{hours} hours and {minutes:02d} minutes"

# Apply the function to calculate the positive difference from 600 minutes
aggregated_baker['Positive Difference from 600'] = aggregated_baker['real Presence Minutes'].apply(calculate_positive_difference)

# Calculate the sum total of the positive differences
sum_total_positive_difference = aggregated_baker['Positive Difference from 600'].sum()

# Convert the sum total from minutes to hours and minutes format
total_difference_hours_minutes = minutes_to_hours(sum_total_positive_difference)

# Print the sum total of the positive differences in minutes and hours:minutes format
print(f"Sum total of the positive difference from 600 minutes: {sum_total_positive_difference} minutes ({total_difference_hours_minutes})")


# Create a DataFrame to display the result as a table
results_df = pd.DataFrame({
    'Total Positive Difference from 600 Minutes': [f"{sum_total_positive_difference} minutes"],
    'Equivalent in Hours and Minutes': [total_difference_hours_minutes]
})

# Display the results as a table
display(results_df)



Sum total of the positive difference from 600 minutes: 1814 minutes (30 hours and 14 minutes)


,Total Positive Difference from 600 Minutes,Equivalent in Hours and Minutes
0,1814 minutes,30 hours and 14 minutes


# **NOW FOR ALL THE "LIGNES"**

In [ ]:
import plotly.express as px

# Function to create histogram for each metric
def create_histogram(df, metric, title, label, sort_by=None, facet_by='Shift'):
    # Sort if a sort column is provided
    if sort_by:
        df = df.sort_values(by=sort_by, ascending=False)

    # Create a histogram
    fig = px.histogram(df, x='Matricule', y=metric, facet_col=facet_by, color=facet_by,
                       title=title, labels={metric: label},
                       category_orders={facet_by: df[facet_by].unique()})
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.update_layout(bargap=0.2)
    return fig

# Loop through each non-null 'Ligne' value
for ligne_value in df['Ligne'].dropna().unique():
    df_ligne = df[df['Ligne'] == ligne_value]

    # Create histograms for each metric
    metrics = {
        'Pause': 'Pause (Minutes)',
        'Breaks': 'Breaks (Count)',
        'real Presence': 'Real Presence (Minutes)'
    }

    for metric, label in metrics.items():
        title = f'{metric} for {ligne_value} Matricule (Total Matricules: {df_ligne["Matricule"].nunique()})'
        fig = create_histogram(df_ligne, metric, title, label, sort_by=metric)
        fig.show()


# **MASTER TABLE**

In [ ]:
from IPython.display import display
import pandas as pd

# Function to check the format and convert to total minutes if needed
def convert_to_minutes(time_data):
    if isinstance(time_data, str):
        hours, minutes = map(int, time_data.split(':'))
        return hours * 60 + minutes
    return time_data

# Loop through each non-null 'Ligne' value
for ligne_value in df['Ligne'].dropna().unique():
    df_ligne = df[df['Ligne'] == ligne_value]

    # Calculate the sums for each 'Matricule'
    aggregated_data = df_ligne.groupby(['Matricule', 'Shift']).agg({
        'Pause': 'sum',
        'Breaks': 'sum',
        'real Presence': 'sum'
    }).reset_index()

    # Convert 'Pause' and 'real Presence' to total minutes if needed
    aggregated_data['Pause'] = aggregated_data['Pause'].apply(convert_to_minutes)
    aggregated_data['real Presence'] = aggregated_data['real Presence'].apply(convert_to_minutes)

    # Add 'Ligne' column
    aggregated_data['Ligne'] = ligne_value

    # Apply conditions for filtering
    condition = (
        (aggregated_data['Pause'] > 120) |
        (aggregated_data['Breaks'] > 2) |
        (aggregated_data['real Presence'] < 600)
    ) & ~(
        (aggregated_data['real Presence'] > 600) &
        (aggregated_data['Breaks'] < 4)
    )

    # Filter and sort the data
    filtered_matricules = aggregated_data[condition]
    sorted_filtered_matricules = filtered_matricules.sort_values(by='real Presence', ascending=True)

    # Count the number of matricules
    num_matricules = sorted_filtered_matricules.shape[0]

    # Print the number of matricules
    print(f"Ligne: {ligne_value}, Number of Employees in the table: {num_matricules}")

    # Display the sorted table in a more readable format
    display(sorted_filtered_matricules)


Ligne: BAKER, Number of Employees in the table: 20


,Matricule,Shift,Pause,Breaks,real Presence,Ligne
12,0599,Jour,465,6,250,BAKER
18,2212,Jour,17,2,295,BAKER
27,6861,Jour,115,4,344,BAKER
40,7069,Jour,322,5,397,BAKER
7,0432,Jour,326,8,405,BAKER
28,6913,Jour,202,4,507,BAKER
38,7032,Jour,183,3,529,BAKER
26,6807,Jour,179,6,536,BAKER
15,2200,Jour,158,3,553,BAKER
17,2211,Jour,140,3,554,BAKER


Ligne: MH, Number of Employees in the table: 31


,Matricule,Shift,Pause,Breaks,real Presence,Ligne
34,7002,Jour,226,4,0,MH
58,7250,Jour,595,2,130,MH
55,7218,Jour,589,3,132,MH
53,7206,Jour,535,7,206,MH
23,6760,Jour,454,6,259,MH
13,0489STG,Jour,434,4,274,MH
26,6914,Jour,243,5,482,MH
1,0087,Jour,212,5,487,MH
35,7002,Nuit,226,4,507,MH
56,7223,Jour,198,4,509,MH


Ligne: MEINCK, Number of Employees in the table: 21


...

,Matricule,Shift,Pause,Breaks,real Presence,Ligne
17,6886,Jour,45,1,0,VRD
13,2205,Jour,491,2,241,VRD
16,2332,Jour,399,7,257,VRD
18,6910,Jour,147,2,346,VRD
7,0124ANA,Jour,326,8,400,VRD
27,7211,Jour,286,8,429,VRD
21,7007,Jour,229,3,464,VRD
20,6953,Jour,204,3,494,VRD
26,7201,Jour,209,8,505,VRD
2,0113ANA,Jour,201,8,511,VRD


Ligne: PATYS, Number of Employees in the table: 19


,Matricule,Shift,Pause,Breaks,real Presence,Ligne
18,6610,Jour,97,2,0,PATYS
32,7136,Nuit,0,0,0,PATYS
12,2187,Jour,421,3,112,PATYS
19,6698,Jour,564,4,159,PATYS
26,6958,Jour,557,4,171,PATYS
10,0488,Jour,529,7,186,PATYS
3,0407STG,Jour,398,5,323,PATYS
25,6923,Nuit,256,2,462,PATYS
14,2193,Jour,173,5,552,PATYS
4,0418,Jour,148,3,562,PATYS


Ligne: CIGARE, Number of Employees in the table: 24


,Matricule,Shift,Pause,Breaks,real Presence,Ligne
23,7076,Jour,629,3,148,CIGARE
34,7188,Jour,559,5,152,CIGARE
18,7014,Jour,561,4,230,CIGARE
15,6840,Jour,450,8,249,CIGARE
36,7190,Jour,360,3,344,CIGARE
32,7170,Jour,184,5,527,CIGARE
14,6837,Jour,173,8,533,CIGARE
26,7123,Jour,170,5,541,CIGARE
30,7139,Jour,172,4,545,CIGARE
13,6831,Jour,157,4,551,CIGARE


# **TOTAL TIME THEFT**

In [ ]:
import pandas as pd
from IPython.display import display

# Define a function to convert "HH:MM" format to minutes
def convert_to_minutes(time_str):
    if isinstance(time_str, str):  # Check if the input is a string
        hours, minutes = map(int, time_str.split(':'))
        return hours * 60 + minutes
    else:
        return time_str  # If it's not a string, return as is

# Define a function to calculate the positive difference only if 'real Presence' is less than 600 minutes
def calculate_positive_difference(real_presence_minutes):
    return max(600 - real_presence_minutes, 0)

# Define a function to convert minutes to hours and minutes format
def minutes_to_hours(minutes):
    hours = minutes // 60
    minutes = minutes % 60
    return f"{hours} hours and {minutes:02d} minutes"

# DataFrame to store the aggregated results for each 'Ligne'
ligne_results = []

# Loop through each unique non-null 'Ligne' value
for ligne_value in df['Ligne'].dropna().unique():
    df_ligne = df[df['Ligne'] == ligne_value].copy()

    # Convert 'real Presence' from "HH:MM" format to minutes
    df_ligne['real Presence Minutes'] = df_ligne['real Presence'].apply(convert_to_minutes)

    # Group by 'Matricule' and sum 'real Presence Minutes'
    aggregated_ligne = df_ligne.groupby('Matricule').agg({
        'real Presence Minutes': 'sum'
    }).reset_index()

    # Apply the function to calculate the positive difference from 600 minutes
    aggregated_ligne['Positive Difference from 600'] = aggregated_ligne['real Presence Minutes'].apply(calculate_positive_difference)

    # Calculate the sum total of the positive differences for this 'Ligne'
    sum_total_positive_difference = aggregated_ligne['Positive Difference from 600'].sum()

    # Convert the sum total to hours and minutes format
    total_difference_hours_minutes = minutes_to_hours(sum_total_positive_difference)

    # Add the results to the DataFrame list
    ligne_results.append({
        'Ligne': ligne_value,
        'Total Positive Difference from 600 Minutes': f"{sum_total_positive_difference} minutes",
        'Equivalent in Hours and Minutes': total_difference_hours_minutes
    })

# Create a DataFrame from the results list
results_df = pd.DataFrame(ligne_results)

# Display the aggregated results for each 'Ligne'
display(results_df)


,Ligne,Total Positive Difference from 600 Minutes,Equivalent in Hours and Minutes
0,BAKER,1814 minutes,30 hours and 14 minutes
1,MH,3026 minutes,50 hours and 26 minutes
2,MEINCK,1483 minutes,24 hours and 43 minutes
3,VRD,2605 minutes,43 hours and 25 minutes
4,PATYS,3537 minutes,58 hours and 57 minutes
5,CIGARE,2394 minutes,39 hours and 54 minutes
